# 说明

思路和7.2中是一样的：都是用文件编写器，把不断变化的指标值写到指定的文件夹中。

In [1]:
import tensorboard as tb
import tensorflow as tf

In [2]:
# 加载数据集：只用训练集即可
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()

# 维度拓展，从数组转为图像尺寸
train_image = tf.expand_dims(train_image, -1)
test_image = tf.expand_dims(test_image, -1)
train_image.shape

# 转换数据类型：
train_image = tf.cast( train_image/255, tf.float32 )  # 归一化后，转为float32
train_label = tf.cast( train_label, tf.int32 )

test_image = tf.cast( test_image/255, tf.float32 )
test_label = tf.cast( test_label, tf.int32 )

In [3]:
# tf.data进行数据集输入：
train_dataset = tf.data.Dataset.from_tensor_slices( (train_image,train_label) )
test_dataset = tf.data.Dataset.from_tensor_slices( (test_image,test_label) )
train_dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int32)>

In [4]:
# 数据集乱序、分批次：因为后面用tf.keras直接搭网络，所以训练数据一般要.repeat()
train_dataset = train_dataset.shuffle(60000).repeat().batch(64)
test_dataset = test_dataset.shuffle(10000).batch(64)

In [5]:
model = tf.keras.Sequential( [
    tf.keras.layers.Conv2D( 16, [3,3], activation = 'relu', input_shape = (None, None, 1) ),
    tf.keras.layers.Conv2D( 32, [3,3], activation = 'relu' ),
    tf.keras.layers.GlobalMaxPooling2D(),  # 换一个Max池化看看（不是重点）
    tf.keras.layers.Dense(10, activation = 'softmax')
] )

### 以下是自定义训练：

In [6]:
# 自定义优化器对象：
optimizer = tf.keras.optimizers.Adam()
# 自定义损失函数对象：
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

In [7]:
# 定义指标计算“对象”：
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_acc')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_acc')

In [ ]:
# 每个batch的训练函数：
def train_step(model, images, labels_real):
    
    with tf.GradientTape() as t:  # 定义t要跟踪哪些函数的梯度变化！
        labels_predict = model(images)                      # 当前model对图像的预测
        loss_step = loss_func(labels_real, labels_predict)  # 直接用
        
    # 梯度优化：
    grads = t.gradient( loss_step, model.trainable_variables )  # 求目标函数关于“各个”可训练参数的梯度值！
    # 用上一步得到“各个”可训练参数梯度值，修改“各个”可训练参数，即也就修改了model
    optimizer.apply_gradients( zip(grads, model.trainable_variables) )  
    
    # 每个batch指标计算：64张训练图片的评价loss和acc —— 公用可调用对象，一直在变！★★★
    train_loss( loss_step )
    train_accuracy( labels_real, labels_predict )

In [ ]:
# 一般每次epoch测试一次就行了，不用每个batch那么频繁！
def test_step(model, images, labels_real):
    labels_predict = model(images)
    loss_step = loss_func(labels_real, labels_predict)
    
    # 每个batch指标计算：64张测试图片的评价loss和acc —— 公用可调用对象，一直在变！★★★
    test_loss( loss_step )
    test_accuracy( labels_real, labels_predict )

In [8]:
# 创建文件编写器：
import os
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")  # 创建一个时间戳

# 文件要写到的磁盘地址：
train_log_dir = 'gradient_tape' + current_time + '/train'
test_log_dir = 'gradient_tape' + current_time + '/test'

# 文件编写器：
train_writer = tf.summary.create_file_writer(train_log_dir)
test_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
# 总体训练的执行函数：注函数！
def train():
    
    # 每个epoch大循环：
    for epoch in range(10):
        # 每个batch小循环：
        for (batch, (images, labels_real)) in enumerate(train_dataset):  # 每次拿一个单位（batch）出来，带上编号
            train_step( model, images, labels_real )  # 每个batch的训练！
            # 在for循环内，也就是每个batch都在打印！
            print( 'Epoch{}_batch{}：train_loss：{}，train_acc：{}'.format(epoch+1, 
                                                                           batch, 
                                                                           train_loss.result().numpy(), 
                                                                           train_accuracy.result().numpy()) )            
        # 一直在累积：内部for循环每完毕一次，就是一个epoch完成，打印
        print('Epoch{}：train_loss：{}，train_acc：{}'.format(epoch+1,
                                                              train_loss.result().numpy(), 
                                                              train_accuracy.result().numpy()) )
        
        # 把每个epoch中的train训练的loss和accuracy写入磁盘：此时train_writer是默认的文件编写器！
        with train_writer.as_default():
            tf.summary.scalar('train_loss', train_loss.result(), step = epoch)
            tf.summary.scalar('train_acc', train_accuracy.result(), step = epoch)
        
        # 每个epoch测试一下当前模型“预测”能力：
        for (batch, (images, labels_real)) in enumerate(test_dataset):
            test_step( model, images, labels_real )
        print('Epoch{}：test_loss：{}，test_acc：{}'.format(epoch+1,
                                                            test_loss.result().numpy(),
                                                            test_accuracy.result().numpy()) )
        
        # 把每个epoch中的test测试的loss和accuracy写入磁盘：此时test_writer是默认的文件编写器！
        with test_writer.as_default():
            tf.summary.scalar('test_loss', test_loss.result(), step = epoch)  # 在tensorboard中显示的标签是test_loss
            tf.summary.scalar('test_acc', test_accuracy.result(), step = epoch)
        
        
        # 每个epoch所有训练集完毕，需要把对象重置：
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [ ]:
train()